# Simple Jorg Spectrum Synthesis

This notebook demonstrates basic spectrum synthesis with Jorg, handling the missing Stark profile data gracefully.

In [ ]:
# Setup
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add Jorg to path
sys.path.insert(0, str(Path('..') / 'src'))

print("✓ JAX setup complete")
print(f"✓ JAX devices: {jax.devices()}")

In [ ]:
# Import Jorg modules
from jorg.synthesis import synth

print("✓ Jorg synthesis module imported")
print("✓ Ready for spectrum synthesis")

In [ ]:
# Test 1: Simple solar spectrum synthesis
print("🌟 Testing Jorg Spectrum Synthesis")
print("=" * 50)

# Solar parameters
stellar_params = {
    'Teff': 5778,     # K
    'logg': 4.44,     # log cm/s²
    'm_H': 0.0,       # [M/H]
    'vsini': 2.0,     # km/s
    'vmic': 1.0       # km/s
}

# Short wavelength range for quick test
wavelength_range = (5050, 5070)  # Å

print(f"Stellar parameters: {stellar_params}")
print(f"Wavelength range: {wavelength_range[0]}-{wavelength_range[1]} Å")

try:
    print(f"\n🔬 Calling Jorg synth()...")
    
    wavelengths, flux, continuum = synth(
        Teff=stellar_params['Teff'],
        logg=stellar_params['logg'],
        m_H=stellar_params['m_H'],
        wavelengths=wavelength_range,
        vsini=stellar_params['vsini'],
        vmic=stellar_params['vmic'],
        rectify=True,
        R=20000  # Lower resolution for faster computation
    )
    
    print(f"\n✅ Synthesis successful!")
    print(f"   Wavelength points: {len(wavelengths)}")
    print(f"   Wavelength range: {wavelengths[0]:.2f} - {wavelengths[-1]:.2f} Å")
    print(f"   Flux range: {flux.min():.3f} - {flux.max():.3f}")
    
    if continuum is not None:
        print(f"   Continuum range: {continuum.min():.2e} - {continuum.max():.2e}")
    
    synthesis_success = True
    
except Exception as e:
    print(f"\n❌ Synthesis failed: {e}")
    print(f"   This may be due to missing data files or incomplete setup")
    import traceback
    traceback.print_exc()
    synthesis_success = False

In [ ]:
# Plot results if synthesis was successful
if synthesis_success:
    plt.figure(figsize=(12, 6))
    
    plt.plot(wavelengths, flux, 'k-', linewidth=1.5, label='Jorg synthetic spectrum')
    plt.xlabel('Wavelength (Å)')
    plt.ylabel('Normalized Flux')
    plt.title(f'Jorg Solar Spectrum: {wavelength_range[0]}-{wavelength_range[1]} Å')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(0.3, 1.1)
    
    # Add some analysis
    deepest_line = flux.min()
    deepest_wavelength = wavelengths[flux.argmin()]
    
    plt.axvline(deepest_wavelength, color='red', linestyle=':', alpha=0.7)
    plt.text(deepest_wavelength, deepest_line - 0.05, 
             f'Deepest line\n{deepest_wavelength:.2f} Å\nDepth: {1-deepest_line:.3f}',
             ha='center', va='top', fontsize=10, 
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Spectrum Analysis:")
    print(f"   Mean flux: {flux.mean():.3f}")
    print(f"   Deepest line: {deepest_line:.3f} at {deepest_wavelength:.2f} Å")
    print(f"   Line depth: {1-deepest_line:.3f}")
    print(f"   Lines detected (< 0.98): {np.sum(flux < 0.98)}")
    
else:
    print("\n⚠️  Synthesis failed - check Jorg installation and data files")
    print("   The Stark profile warning is expected and should not prevent synthesis")
    print("   If synthesis fails, check:")
    print("   1. Line list data availability")
    print("   2. Atmosphere model files")
    print("   3. Partition function data")

In [ ]:
# Test 2: Continuum-only synthesis (if full synthesis fails)
if not synthesis_success:
    print(f"\n🧪 Testing continuum-only synthesis...")
    
    try:
        from jorg.continuum.core import total_continuum_absorption
        from jorg.abundances import calculate_eos_with_asplund
        from jorg.constants import SPEED_OF_LIGHT
        
        # Simple test parameters
        temperature = 5778  # K
        total_density = 1e16  # cm^-3
        electron_guess = 1e13  # cm^-3
        
        # Calculate chemical equilibrium
        electron_density, number_densities = calculate_eos_with_asplund(
            temperature, total_density, electron_guess, 0.0
        )
        
        # Test wavelengths
        test_wavelengths = np.linspace(5000, 5100, 100)  # Å
        frequencies = SPEED_OF_LIGHT / (test_wavelengths * 1e-8)
        
        # Simple partition functions
        partition_functions = {
            'H_I': lambda log_T: 2.0,
            'He_I': lambda log_T: 1.0
        }
        
        # Calculate continuum opacity
        continuum_opacity = total_continuum_absorption(
            jnp.array(frequencies),
            temperature,
            electron_density,
            number_densities,
            partition_functions,
            include_metals=True
        )
        
        print(f"✅ Continuum calculation successful!")
        print(f"   Electron density: {electron_density:.2e} cm^-3")
        print(f"   Species calculated: {len(number_densities)}")
        print(f"   Opacity range: {continuum_opacity.min():.2e} - {continuum_opacity.max():.2e} cm^-1")
        
        # Plot continuum opacity
        plt.figure(figsize=(10, 6))
        plt.semilogy(test_wavelengths, continuum_opacity, 'b-', linewidth=2)
        plt.xlabel('Wavelength (Å)')
        plt.ylabel('Continuum Opacity (cm⁻¹)')
        plt.title('Jorg Continuum Opacity (includes metal bound-free)')
        plt.grid(True, alpha=0.3)
        plt.show()
        
    except Exception as e:
        print(f"❌ Continuum test also failed: {e}")
        import traceback
        traceback.print_exc()

## Notes on Warnings

### ✅ Expected Warnings
- **Stark profile data missing**: This is expected and won't prevent synthesis
- **Metal BF data loaded**: This confirms Jorg is working correctly

### 🔧 If Synthesis Fails
1. **Missing line lists**: Jorg may need VALD or Kurucz line data
2. **Missing atmosphere models**: MARCS model files may be needed
3. **Incomplete setup**: Some data files may not be fully configured

### 🚀 Next Steps
- For production use, download complete Korg.jl data directory
- Copy Stark profile data from Korg.jl installation
- Add line list files for comprehensive synthesis